In [76]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os
from collections import Counter

In [73]:
def seismic_ascii_parser(file_path):
    all_data = []
    head = ""
    xarray = []
    yarray = []
    zarray = []
    with open(file_path, 'r') as f:
        content = f.readlines()
        for line in content:
            if '#' in line:
                head += line
                continue
            l = line.split(' ')
            xarray.append(float(l[0]))
            yarray.append(float(l[1]))
            zarray.append(float(l[2]))

    df = pd.DataFrame({'x': xarray, 'y': yarray, 'z': zarray})
    all_data.append({"test": df})
    return create_seismic_ascii_date(all_data)[0]

def create_seismic_ascii_date(data_frame):
    for dataframe_data in data_frame:
        _, value = next(iter(dataframe_data.items()))
        return value.to_dict("records"),


In [84]:
Lateral_trend = seismic_ascii_parser(r"C:/HV/Seismic/datas/test_3_5/Lateral_trend_No_smooth_Heff.txt")

In [90]:
print(Lateral_trend[:10])

[{'x': 445946.632991, 'y': 7004198.214381, 'z': 72.554604}, {'x': 445996.632991, 'y': 7004198.214381, 'z': 74.0868}, {'x': 446046.632991, 'y': 7004198.214381, 'z': 74.0868}, {'x': 446096.632991, 'y': 7004198.214381, 'z': 75.619003}, {'x': 446146.632991, 'y': 7004198.214381, 'z': 74.0868}, {'x': 446196.632991, 'y': 7004198.214381, 'z': 71.0224}, {'x': 446246.632991, 'y': 7004198.214381, 'z': 72.554604}, {'x': 446296.632991, 'y': 7004198.214381, 'z': 72.554604}, {'x': 446346.632991, 'y': 7004198.214381, 'z': 74.0868}, {'x': 446396.632991, 'y': 7004198.214381, 'z': 72.554604}]


In [91]:
df_Lateral_trend = pd.DataFrame(Lateral_trend)

In [85]:
column_names = ['wells', 'name', 'inlines', 'xlines', 'top', 'bottom'] 
df_all_wells_coor = pd.read_csv('C:/HV/Seismic/datas/test_3_5/data_wells.txt', sep='\t', names=column_names, header=None)

In [86]:
df_all_wells_coor

,wells,name,inlines,xlines,top,bottom
1,660R,TPU_BVBJ1_1_TOP_S,450928.636400,7.011087e+06,2761.150000,2874.640
2,661R,TPU_BVBJ1_1_TOP_S,457638.951200,7.011792e+06,2769.376000,2898.366
3,662R,TPU_BVBJ1_1_TOP_S,455158.133924,7.014716e+06,2775.608000,2888.708
4,663R,TPU_BVBJ1_1_TOP_S,447671.940000,7.012181e+06,2771.639000,2875.099
5,669R,TPU_BVBJ1_1_TOP_S,451334.990000,7.011850e+06,2759.749000,2870.559
6,673R,TPU_BVBJ1_1_TOP_S,449194.772480,7.008929e+06,2765.822834,2893.166
7,676R,TPU_BVBJ1_1_TOP_S,454672.154278,7.009559e+06,2776.051783,2915.217


In [87]:

def process_txt_files(folder_path):
    # Список для хранения данных
    data = []

    # Проход по всем файлам в указанной папке
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            # Получаем имя файла до символа "_"
            name_part = filename.split('_')[0]

            # Полный путь к файлу
            file_path = os.path.join(folder_path, filename)

            # Чтение файла
            with open(file_path, 'r') as file:
                # Пропускаем строки до "~Ascii"
                for line in file:
                    if line.startswith('~Ascii'):
                        break

                # Читаем оставшиеся строки и собираем данные
                values = []
                for line in file:
                    # Разделяем строки на значения
                    parts = line.split()
                    if len(parts) >= 2:
                        value = parts[1]
                        if value != '-999.250000':
                            values.append(float(value))

                # Подсчет значений
                if values:
                    value_counts = Counter(values)
                    # Находим наиболее частое значение
                    most_common_value, most_common_count = value_counts.most_common(1)[0]

                    # Создаем словарь для хранения данных
                    entry = {
                        'имя': name_part,
                        'наиболее_частое_значение': most_common_value,
                        'количество': most_common_count
                    }

                    # Добавляем остальные значения и их количество
                    for value, count in value_counts.items():
                        if value != most_common_value:
                            entry[f'значение_{value}'] = value
                            entry[f'количество_{value}'] = count

                    data.append(entry)

    # Создаем DataFrame из собранных данных
    result_df = pd.DataFrame(data)

    # Заполняем пропуски нулями для удобства
    result_df.fillna(0, inplace=True)

    return result_df

# Пример использования функции
folder_path = 'C:/HV/Seismic/datas/test_3_5/Log_facii'   # Укажите путь к вашей папке
df_facii  = process_txt_files(folder_path)
print(df_facii )


    имя  наиболее_частое_значение  количество  значение_1.0  количество_1.0
0  660R                       0.0        1697           1.0             933
1  661R                       0.0         703           1.0             363
2  662R                       0.0        1210           1.0             440
3  663R                       0.0        1080           1.0             557
4  669R                       0.0         918           1.0             213
5  673R                       0.0         825           1.0             252
6  676R                       0.0         654           1.0             326


In [92]:
import pandas as pd


# Шаг 1: Сортируем датафреймы по значениям для использования merge_asof
df_Lateral_trend = df_Lateral_trend.sort_values('x')
df_all_wells_coor = df_all_wells_coor.sort_values('inlines')

# Шаг 2: Объединяем Lateral_trend и df_all_wells_coor с использованием merge_asof
merged_df = pd.merge_asof(df_Lateral_trend, df_all_wells_coor, left_on='x', right_on='inlines', direction='nearest')

# Выводим промежуточный результат
print("После первого объединения:")
print(merged_df)

# Шаг 3: Объединяем с df_facii по колонке wells
df_result = pd.merge(merged_df, df_facii, left_on='wells', right_on='имя', how='inner')

# Выводим промежуточный результат
print("После второго объединения:")
print(df_result)

# Шаг 4: Создаем итоговый датафрейм с нужными колонками
df_result_final = df_result[['wells', 'z', 'наиболее_частое_значение']]
df_result_final.columns = ['Скважина', 'класс по карте', 'класс по скважине']

# Выводим финальный результат
print("Итоговый DataFrame:")
print(df_result_final)


После первого объединения:
                   x             y          z wells               name  \
0      445946.632991  7.004198e+06  72.554604  663R  TPU_BVBJ1_1_TOP_S   
1      445946.632991  7.011948e+06  75.619003  663R  TPU_BVBJ1_1_TOP_S   
2      445946.632991  7.011998e+06  77.151199  663R  TPU_BVBJ1_1_TOP_S   
3      445946.632991  7.012048e+06  80.215599  663R  TPU_BVBJ1_1_TOP_S   
4      445946.632991  7.012098e+06  83.279999  663R  TPU_BVBJ1_1_TOP_S   
...              ...           ...        ...   ...                ...   
80075  460196.632991  7.013548e+06  80.215599  661R  TPU_BVBJ1_1_TOP_S   
80076  460196.632991  7.013598e+06  78.683403  661R  TPU_BVBJ1_1_TOP_S   
80077  460196.632991  7.013648e+06  80.215599  661R  TPU_BVBJ1_1_TOP_S   
80078  460196.632991  7.013348e+06  81.747803  661R  TPU_BVBJ1_1_TOP_S   
80079  460196.632991  7.018148e+06  80.215599  661R  TPU_BVBJ1_1_TOP_S   

           inlines        xlines       top    bottom  
0      447671.9400  7.012181e

In [94]:
import pandas as pd

# Примерные данные для Lateral_trend
data_lateral_trend = [
    {'x': 445946.632991, 'y': 7004198.214381, 'z': 72.554604},
    {'x': 445996.632991, 'y': 7004198.214381, 'z': 74.0868},
    {'x': 446046.632991, 'y': 7004198.214381, 'z': 74.0868},
    {'x': 446096.632991, 'y': 7004198.214381, 'z': 75.619003},
    {'x': 446146.632991, 'y': 7004198.214381, 'z': 74.0868},
    {'x': 446196.632991, 'y': 7004198.214381, 'z': 71.0224},
    {'x': 446246.632991, 'y': 7004198.214381, 'z': 72.554604},
    {'x': 446296.632991, 'y': 7004198.214381, 'z': 72.554604},
    {'x': 446346.632991, 'y': 7004198.214381, 'z': 74.0868},
    {'x': 446396.632991, 'y': 7004198.214381, 'z': 72.554604}
]
Lateral_trend = pd.DataFrame(data_lateral_trend)

# Примерные данные для df_all_wells_coor
data_all_wells_coor = {
    'wells': ['Well1', 'Well2', 'Well3'],
    'name': ['A', 'B', 'C'],
    'inlines': [445950, 446100, 446300],
    'xlines': [7004200, 7004200, 7004200],
    'top': [10, 20, 30],
    'bottom': [15, 25, 35]
}
df_all_wells_coor = pd.DataFrame(data_all_wells_coor)

# Приводим тип данных inlines к float
df_all_wells_coor['inlines'] = df_all_wells_coor['inlines'].astype(float)

# Шаг 1: Сортируем датафреймы по значениям для использования merge_asof
Lateral_trend = Lateral_trend.sort_values('x')
df_all_wells_coor = df_all_wells_coor.sort_values('inlines')

# Шаг 2: Объединяем Lateral_trend и df_all_wells_coor с использованием merge_asof
merged_df = pd.merge_asof(Lateral_trend, df_all_wells_coor, left_on='x', right_on='inlines', direction='nearest')

# Шаг 3: Удаляем дубликаты, оставляя только одно совпадение
merged_df = merged_df.drop_duplicates(subset='x')

# Выводим промежуточный результат
print("Результат объединения с уникальными совпадениями:")
print(merged_df)


Результат объединения с уникальными совпадениями:
               x             y          z  wells name   inlines   xlines  top  \
0  445946.632991  7.004198e+06  72.554604  Well1    A  445950.0  7004200   10   
1  445996.632991  7.004198e+06  74.086800  Well1    A  445950.0  7004200   10   
2  446046.632991  7.004198e+06  74.086800  Well2    B  446100.0  7004200   20   
3  446096.632991  7.004198e+06  75.619003  Well2    B  446100.0  7004200   20   
4  446146.632991  7.004198e+06  74.086800  Well2    B  446100.0  7004200   20   
5  446196.632991  7.004198e+06  71.022400  Well2    B  446100.0  7004200   20   
6  446246.632991  7.004198e+06  72.554604  Well3    C  446300.0  7004200   30   
7  446296.632991  7.004198e+06  72.554604  Well3    C  446300.0  7004200   30   
8  446346.632991  7.004198e+06  74.086800  Well3    C  446300.0  7004200   30   
9  446396.632991  7.004198e+06  72.554604  Well3    C  446300.0  7004200   30   

   bottom  
0      15  
1      15  
2      25  
3      25 

In [78]:
#test
df = pd.read_excel('C:/HV/Seismic/datas/SFA_SO.xlsx')
df

,Скважина,Сейсм класс,Скв класс1
0,9,5,1
1,16,4,1
2,8,5,3
3,3,5,2
4,2,5,1
5,17,4,1
6,7,5,2
7,5,5,2
8,14,4,1
9,15,4,1


In [22]:


def plot_with_trendline(df, x_col, y_col, name_col):
    """
    Функция для построения графика с линией тренда и значением R^2,
    а также отображением всех имен при наведении на точки.

    :param df: DataFrame, содержащий данные
    :param x_col: str, имя колонки для оси X
    :param y_col: str, имя колонки для оси Y
    :param name_col: str, имя колонки с именами для отображения
    """
    # Преобразование значений в колонке с именами в строковый формат
    df[name_col] = df[name_col].astype(str)

    # Создание новой колонки с объединенными именами для одинаковых координат
    df['hover_names'] = df.groupby([x_col, y_col])[name_col].transform(lambda x: ', '.join(x))

    # Извлечение данных
    x = df[x_col]
    y = df[y_col]
    hover_names = df['hover_names']

    # Вычисление коэффициентов линейной регрессии
    coefficients = np.polyfit(x, y, 1)
    polynomial = np.poly1d(coefficients)
    trendline = polynomial(x)

    # Вычисление R^2
    r_squared = 1 - (np.sum((y - trendline) ** 2) / np.sum((y - np.mean(y)) ** 2))

    # Форматирование уравнения линии тренда
    slope = coefficients[0]
    intercept = coefficients[1]
    equation = f'y = {slope:.2f}x + {intercept:.2f}'


    # Создание графика
    fig = go.Figure()

    # Добавление точки данных с текстом для отображения имен
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers',
        name='Скважины',
        text=hover_names,  # Текст для отображения при наведении
        hoverinfo='text'  # Отображение только текста
    ))

    # Добавление линии тренда
    fig.add_trace(go.Scatter(x=x, y=trendline, mode='lines', name='Линия тренда'))

    # Добавление аннотации с R^2
    fig.add_annotation(
        x=0.5,
        y=0.9,
        text=f'R² = {r_squared:.2f}<br>{equation}',
        showarrow=False,
        font=dict(size=16),
        xref="paper",
        yref="paper"
    )

    # Настройки графика
    fig.update_layout(
        title='Скважины с линией тренда',
        xaxis_title=x_col,
        yaxis_title=y_col
    )

    # Показать график
    fig.show()


In [23]:
plot_with_trendline(df, 'Сейсм класс', 'Скв класс1', 'Скважина')

In [24]:
df

,Скважина,Сейсм класс,Скв класс1,hover_names
0,9,5,1,"9, 2"
1,16,4,1,"16, 17, 14, 15, 4"
2,8,5,3,8
3,3,5,2,"3, 7, 5"
4,2,5,1,"9, 2"
5,17,4,1,"16, 17, 14, 15, 4"
6,7,5,2,"3, 7, 5"
7,5,5,2,"3, 7, 5"
8,14,4,1,"16, 17, 14, 15, 4"
9,15,4,1,"16, 17, 14, 15, 4"
